<a href="https://colab.research.google.com/github/Kaiziferr/machine_learning/blob/main/knn/01_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install plotly

In [10]:
import warnings
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score


# **Información**
## **Datos**
---
Pima Indians Diabetes

Conjunto de datos proviene del Instituto Nacional de Diabetes y Enfermedades Digestivas y Renales. El objetivo del conjunto de datos es predecir de forma diagnóstica si un paciente tiene diabetes o no, basándose en ciertas mediciones de diagnóstico incluidas en el conjunto de datos. Se impusieron varias restricciones a la selección de estas instancias de una base de datos más grande.

**Información de la Entidad**
- Institutos Nacionales de Diabetes y Enfermedades Digestivas y Renales

---
## **Proceso**
**Proposito**
- Explorar el KNN
- Predecir quién puede presentar diabetes en mujeres con ascendencia Pima

**Diccionario de datos**
- Pregnancies: Número de veces embarazada
- Glucose: Concentración de glucosa plasmática a las 2 horas en una prueba de tolerancia oral a la glucosa.
- BloodPressure: Presión arterial diastólica (mm Hg)
- SkinThickness: Grosor del pliegue cutáneo del tríceps (mm)
- Insulin: Insulina sérica de 2 horas (mu U/ml)
- BMI: Índice de masa corporal (peso en kg/(altura en m)^2)
- DiabetesPedigreeFunction: Función del pedigrí de la diabetes
- Age: Años de edad
- Outcome: Variable de clase (0 o 1) 268 de 768 son 1, los demás son 0



# **Config**
---



In [11]:
pd.set_option('display.float_format', '{:,.2f}'.format)
title_data = 'Pima Indians Diabetes'
warnings.filterwarnings("ignore")
random_seedd = 12354

# **Data**
---

In [12]:
url_data = 'https://drive.google.com/file/d/1FGFPdobSr2bMdYJOou-qWV8pJ1iOiW5d/view?usp=sharing'
url_data = 'https://drive.google.com/uc?id=' + url_data.split('/')[-2]
data = pd.read_csv(url_data, dtype='str')

In [13]:
columns_data = data.columns[:].to_list()

In [14]:
dfs = data.groupby(['Outcome']).count().reset_index()
dfs

,Outcome,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0,500,500,500,500,500,500,500,500
1,1,268,268,268,268,268,268,268,268


In [15]:
fig = px.bar(
    data,
    x='Outcome',
    title=title_data,
    labels={
        'count':'Conteo dibetes',
        'Outcome': 'Tiene diabetes'},
    color='Outcome')

fig.add_trace(go.Scatter(
    x=dfs.index,
    y=dfs['Age'],
    text=dfs['Age'],
    mode='text',
    textposition='top center',
    textfont=dict(
        size=18,
    ),
    showlegend=False
))

fig.show()

El conjunto tiene un desbalance considerable, entre aquellos registros que detectan diabetes, y aquellos que no.

# **Asignacion tipo de dato**

In [16]:
data[columns_data[:-4] + columns_data[-2:] ] = data[columns_data[:-4] + columns_data[-2:] ].astype("int32")
data[columns_data[-4:-2]] = data[columns_data[-4:-2]].astype("float32")

- Se presenta un desbalance en los datos

# **Data Split**
---

In [17]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [18]:
random_seedd

12354

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    train_size=0.8,
    shuffle=True,
    stratify=y,
    random_state=random_seedd)

# **Normalizaciòn**
---

In [20]:
estimator = StandardScaler()
estimator.fit(X_train)
X_train = estimator.transform(X_train)
X_test = estimator.transform(X_test)

- Se aplica una estandarización simple (puede que no sea la mejor estimación), ya que este es un proceso de exploración del modelo KNN

# **Model**
---

In [21]:
model_base = KNeighborsClassifier()
model_base.fit(X_train, y_train)

KNeighborsClassifier()

In [22]:
print(f'Tasa de prediciones en el conjunto de entrenamiento: {model_base.score(X_train, y_train)}')
print(f'Tasa de prediciones en el conjunto de testing: {model_base.score(X_test, y_test)}')

Tasa de prediciones en el conjunto de entrenamiento: 0.8273615635179153
Tasa de prediciones en el conjunto de testing: 0.7337662337662337


El modelo tiene una tasa de aciertos de aproximadamente del 83% sobre los datos de entrenamiento, mientras que sobre los datos de validación la tasa de aciertos es aproximadamente del 73.4%, no se evidencia sobreajuste, sin embargo, como el problema implica la detección de una enfermedad, se requiere de una métrica más robusta.

## **Seleccion del mejor K**
---

In [24]:
n_neighbors, train_scores, test_scores = [], [], []

for k in range(1, 40, 5):
    n_neighbors.append(k)
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    train_scores.append(model.score(X_train, y_train))
    test_scores.append(model.score(X_test, y_test))

In [25]:
scoring = pd.DataFrame(
    {
        'K': n_neighbors,
        'train_score':train_scores,
        'test_score':test_scores
    }
)

In [26]:
fig = px.line(
    scoring,
    x='K',
    y=['train_score','test_score'],
    title='Mejor K'+title_data,
    markers=True)

fig.update_traces(textposition="bottom right")


- Lo que se evidencia es que el modelo no tiene un buen rendimiento con los datos de validación, ya que con ningún k supera el rendimiento con los datos de entrenamiento.
- Es se puede presentar porque los datos de validación no son significativos para el proceso, tocaría probar con una nueva división de los datos.
- Otro factor es por una mala configuración de los hiperparámetros
Mala selección de características o falta de ingeniería de características que permitan extraer mejor la información
- Un proceso de pipeline no adecuado como la normalización
-El modelo no necesariamente presenta desajuste, puesto que su rendimiento en su mayoría está por encima del 70%, ahora el accuracy no es la métrica idónea.

In [27]:
n_neighbors, train_scores, test_scores = [], [], []

for k in range(1, 40, 5):
    n_neighbors.append(k)
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    y_predict_train = model.predict(X_train)
    y_predict_test = model.predict(X_test)
    train_scores.append(
        recall_score(y_train, y_predict_train, pos_label=1, average='binary'))
    test_scores.append(
        recall_score(y_test, y_predict_test, pos_label=1, average='binary'))

In [28]:
scoring_recall = pd.DataFrame(
    {
        'K': n_neighbors,
        'train_score':train_scores,
        'test_score':test_scores
    }
)

In [29]:
fig = px.line(
    scoring_recall,
    x='K',
    y=['train_score','test_score'],
    title='Mejor K'+title_data,
    markers=True)

fig.update_traces(textposition="bottom right")

Se empleó un proceso de selección del mejor k con la métrica de recall, que permite calcular el rendimiento del modelo frente al suceso del caso de interés (diabetes), cuando este ocurre. El modelo tiene un rendimiento regular, a diferencia de los K 10 y k 11, incluso este último supera el comportamiento del proceso con los datos de entrenamiento.

In [30]:
param_grid = {
    'n_neighbors': np.arange(1, 10, 2),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
}
model = KNeighborsClassifier()
grid = GridSearchCV(model, param_grid, cv=10, refit=True, scoring = 'recall')

In [31]:
grid.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': array([1, 3, 5, 7, 9]),
                         'weights': ['uniform', 'distance']},
             scoring='recall')

In [32]:
print('Mejor: %f usando %s' % (grid.best_score_, grid.best_params_))

Mejor: 0.545022 usando {'algorithm': 'auto', 'n_neighbors': 3, 'weights': 'uniform'}


In [33]:
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']

for mean, std, param in zip(means, stds, params):
  print('%f (%f) con %r' % (mean, std, param))

0.513420 (0.055975) con {'algorithm': 'auto', 'n_neighbors': 1, 'weights': 'uniform'}
0.513420 (0.055975) con {'algorithm': 'auto', 'n_neighbors': 1, 'weights': 'distance'}
0.545022 (0.118897) con {'algorithm': 'auto', 'n_neighbors': 3, 'weights': 'uniform'}
0.545022 (0.118897) con {'algorithm': 'auto', 'n_neighbors': 3, 'weights': 'distance'}
0.531602 (0.117276) con {'algorithm': 'auto', 'n_neighbors': 5, 'weights': 'uniform'}
0.531602 (0.117276) con {'algorithm': 'auto', 'n_neighbors': 5, 'weights': 'distance'}
0.517749 (0.116102) con {'algorithm': 'auto', 'n_neighbors': 7, 'weights': 'uniform'}
0.517749 (0.116102) con {'algorithm': 'auto', 'n_neighbors': 7, 'weights': 'distance'}
0.494372 (0.131263) con {'algorithm': 'auto', 'n_neighbors': 9, 'weights': 'uniform'}
0.508225 (0.134240) con {'algorithm': 'auto', 'n_neighbors': 9, 'weights': 'distance'}
0.513420 (0.055975) con {'algorithm': 'ball_tree', 'n_neighbors': 1, 'weights': 'uniform'}
0.513420 (0.055975) con {'algorithm': 'ball_

In [34]:
model_final = grid.best_estimator_
model_final.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 3,
 'p': 2,
 'weights': 'uniform'}

In [35]:
print(f'Tasa de prediciones correctas en el conjunto de entrenamiento: {model_final.score(X_train, y_train)}')
print(f'Tasa de prediciones correctas en el conjunto de testing: {model_final.score(X_test, y_test)}')

Tasa de prediciones correctas en el conjunto de entrenamiento: 0.8501628664495114
Tasa de prediciones correctas en el conjunto de testing: 0.7467532467532467


In [37]:
y_predict_train_final = model_final.predict(X_train)
y_predict_train_final = recall_score(y_train, y_predict_train_final, pos_label=1, average="binary")
y_predict_test_final = model_final.predict(X_test)
y_predict_test_final = recall_score(y_test, y_predict_test_final, pos_label=1, average="binary")

In [38]:
print(f'Tasa de prediciones cuando la clase de interes ocurre (diabetes) en el conjunto de entrenamiento: {y_predict_train_final}')
print(f'Tasa de prediciones cuando la clase de interes ocurre (diabetes) en el conjunto de testing: {y_predict_test_final}')

Tasa de prediciones cuando la clase de interes ocurre (diabetes) en el conjunto de entrenamiento: 0.7102803738317757
Tasa de prediciones cuando la clase de interes ocurre (diabetes) en el conjunto de testing: 0.6481481481481481


Su utilizo un cross validation con una malla para seleccionar los mejores hiperparámetros para el modelo, el cual tiene una mejora en la métrica de sensibilidad sobre los datos de validación de aproximadamente el 65%, 4% frente a un modelo con los datos por defecto.

El modelo como está no es el más adecuado para la detección del diabetes en la comunidad, es posible que con más configuraciones el modelo pueda tener una mejora; sin embargo, es recomendable un modelo más complejo tipo instancia.


Recordemos que este es un experimento para evaluar configuraciones del knn e ir aprendiendo ploty